# FireCrawl の Extract API を使ってみる

AI エージェントを作る際、多くの場合何らかの外部のデータをウェブ経由で取得する必要があります。

その際に、FireCrawl の Extract API を使うと、ウェブページのテキストをかなり簡単に取得できます。

Firecrawl を使うメリット：

- 自らクローラーの実装や運用を行う必要がない
- ウェブページ上での細かな仕様変更に対応する必要がない

API をたくさん叩く必要がある際にはコスト感も気になってきそうではありますが、
その分スクレイピング周りの開発を行わなくて済むのでかなりメリットがありそうです。
また、IP アドレスのローテーションなどもしてくれる点も、API を使うメリットですね。

（今後 AI エージェントが増えてくると、ボット対策をするサイトと、AI アクセスを推奨するサイトとで 2 極化が進みそうです。
もっというと、直近しばらくはボット対策をどう迂回するかが、エージェントを作る側の身としては課題となっていくかと思います。）

これまであった API とは別に `extract` という API が最近リリースされ非常に便利だったので、紹介していきます。

## 使い方

Firecrawl のライブラリは下記でインストールできます。

`pip install firecrawl-py`

次に、`.env` ファイルを作って、 `FIRECRAWL_API_KEY` をセットします。

```
FIRECRAWL_API_KEY=your_api_key
```

## 主に使うメソッド

- `scrape_url`: ウェブページをスクレイピングして、そこから必要なテキストを取得する
- `extract`: スクレイピングしたテキストを、指定したスキーマに沿って抽出する

一個ずつ見ていきましょう。


In [4]:
from dotenv import load_dotenv
from firecrawl import FirecrawlApp

load_dotenv(override=True)

target_url = "https://news.yahoo.co.jp/articles/7de60db6f536a3f9021fec6b976edc1fbf438357"

firecrawl = FirecrawlApp()

## Firecrawl の scrape_url の使い方

scrape_url 関数は引数に URL と、フォーマットを指定します。

フォーマットは html, markdown, json が指定できます。


In [6]:
res = firecrawl.scrape_url(target_url, {"formats": ["markdown"]})

print(res["markdown"])

エラーコード：1001100

ご利用の環境では映像を視聴できません。

映像視聴における推奨環境はこちらをご確認ください。

All Nippon NewsNetwork(ANN)

中国東部の浙江省杭州市に本社を置く新興企業「ディープシーク（DeepSeek）」が最新の人工知能（AI）モデルを発表し、世界の株式市場と米政権を揺るがした。同社製品は、米国のトップモデルに性能面で引けを取らず、低コストで実現したとしている。AI半導体大手の米エヌビディアなどの株価急落に直結した。この衝撃を受けて、1月27日は、世界的にハイテク株が売られた。これにより、エヌビディアは、時価総額で一時18％下落し、5927億ドル（約92兆円）が消失した。トランプ米大統領は27日、「ディープシークによる生成AIの発表が、競争に勝つためにさらに集中すべきだという米国AI業界への警鐘として受け止めるべきだ」と語った。

米シリコンバレーのベンチャーキャピタリスト、マーク・アンドリーセン氏は1月26日のSNS「X」で、ディープシークの「R1モデル」は、AIの「スプートニク・モーメント」だと投稿した。アンドリーセン氏は、1950年代後半に、宇宙競争の開始を告げた旧ソビエト連邦による人工衛星の打ち上げに例え、米国が抱く危機感を表した。米紙「ワシントンポスト」によると、今回の開発費は約560万ドル（約8．7億円）で、米国製の1割程度とされる。ディープシークには、エヌビディア製の格落ちチップ「H800」 約2000個が、一方、米国製は最新型「H100」数万個が使用されている。

主要テック企業3社は1月21日、米国におけるAIインフラの拡大を目指す新会社「スターゲート」を設立すると発表した。オープンAIのサム・アルトマン最高経営責任者（CEO）、ソフトバンクの孫正義CEO、オラクルのラリー・エリソン会長は同日、ホワイトハウスでトランプ大統領と共に設立を発表。同プロジェクトには今後、米国内で最大5000億ドル（約78兆円）の投資が予定されている。トランプ氏は、「世界的なテックの巨人が『スターゲート』を設立する。技術とAIは全部”Made in USA”にする」と述べた。計画には、全米20カ所のデータセンター建設も含まれている。最初となる約50万平方フィートのデータセンターは、米テキサス州に建設する予定とな

scrape_url の json モードでは json 形式のスキーマが指定できます。試してみましょう。


In [7]:
from pydantic import BaseModel


class NewsArticle(BaseModel):
    title: str
    content: str
    summary: str


scrape_result = firecrawl.scrape_url(
    target_url,
    {
        "formats": ["json"],
        "jsonOptions": {"schema": NewsArticle.model_json_schema()},
    },
)

print(scrape_result["json"])

{'title': 'NewsArticle', 'content': '中国東部の浙江省杭州市に本社を置く新興企業「ディープシーク（DeepSeek）」が最新の人工知能（AI）モデルを発表し、世界の株式市場と米政権を揺るがした。同社製品は、米国のトップモデルに性能面で引けを取らず、低コストで実現したとしている。AI半導体大手の米エヌビディアなどの株価急落に直結した。この衝撃を受けて、1月27日は、世界的にハイテク株が売られた。これにより、エヌビディアは、時価総額で一時18％下落し、5927億ドル（約92兆円）が消失した。トランプ米大統領は27日、「ディープシークによる生成AIの発表が、競争に勝つためにさらに集中すべきだという米国AI業界への警鐘として受け止めるべきだ」と語った。\n\n米シリコンバレーのベンチャーキャピタリスト、マーク・アンドリーセン氏は1月26日のSNS「X」で、ディープシークの「R1モデル」は、AIの「スプートニク・モーメント」だと投稿した。アンドリーセン氏は、1950年代後半に、宇宙競争の開始を告げた旧ソビエト連邦による人工衛星の打ち上げに例え、米国が抱く危機感を表した。米紙「ワシントンポスト」によると、今回の開発費は約560万ドル（約8．7億円）で、米国製の1割程度とされる。ディープシークには、エヌビディア製の格落ちチップ「H800」 約2000個が、一方、米国製は最新型「H100」数万個が使用されている。\n\n主要テック企業3社は1月21日、米国におけるAIインフラの拡大を目指す新会社「スターゲート」を設立すると発表した。オープンAIのサム・アルトマン最高経営責任者（CEO）、ソフトバンクの孫正義CEO、オラクルのラリー・エリソン会長は同日、ホワイトハウスでトランプ大統領と共に設立を発表。同プロジェクトには今後、米国内で最大5000億ドル（約78兆円）の投資が予定されている。トランプ氏は、「世界的なテックの巨人が『スターゲート』を設立する。技術とAIは全部”Made in USA”にする」と述べた。計画には、全米20カ所のデータセンター建設も含まれている。最初となる約50万平方フィートのデータセンターは、米テキサス州に建設する予定となっている。\n\n米ホワイトハウスのレビット報道官は1月28日、ディープシークが国家安全保障に与える影響

ページに関する細かいメタデータもついてきます。


In [24]:
scrape_result

{'metadata': {'twitter:image': 'https://newsatcl-pctr.c.yimg.jp/t/amd-img/20250202-00000038-ann-000-5-thumb.jpg?exp=10800',
  'twitter:description': '中国東部の浙江省杭州市に本社を置く新興企業「ディープシーク（DeepSeek）」が最新の人工知能（AI）モデルを発表し、世界の株式市場と米政権を揺るがした。同社製品は、米国のトップモデルに性能面で引',
  'al:ios:app_name': 'Yahoo!ニュース',
  'msvalidate.01': '23622BF224ED75EAF67061E9A6003026',
  'twitter:app:name:iphone': 'Yahoo!ニュース',
  'og:image:width': '640',
  'twitter:app:id:iphone': '407906756',
  'twitter:app:id:googleplay': 'jp.co.yahoo.android.news',
  'og:description': '中国東部の浙江省杭州市に本社を置く新興企業「ディープシーク（DeepSeek）」が最新の人工知能（AI）モデルを発表し、世界の株式市場と米政権を揺るがした。同社製品は、米国のトップモデルに性能面で引',
  'ogImage': 'https://newsatcl-pctr.c.yimg.jp/t/amd-img/20250202-00000038-ann-000-5-thumb.jpg?exp=10800',
  'og:locale': 'ja_JP',
  'favicon': 'https://s.yimg.jp/c/icon/s/bsc/2.0/favicon.ico',
  'google-site-verification': 'dVyTbsDzUrUAjCE1aCH9hKwZ8sOsCsvi2uhuBvasrp8',
  'og:title': '【中国AIディープシーク】低コストで高性能“米政権に焦燥”技術覇権と安全保障は？（テレビ朝日系（ANN）） - Yahoo!ニュース',
  'title': '

In [10]:
scrape_result["json"]

{'title': 'NewsArticle',
 'content': '中国東部の浙江省杭州市に本社を置く新興企業「ディープシーク（DeepSeek）」が最新の人工知能（AI）モデルを発表し、世界の株式市場と米政権を揺るがした。同社製品は、米国のトップモデルに性能面で引けを取らず、低コストで実現したとしている。AI半導体大手の米エヌビディアなどの株価急落に直結した。この衝撃を受けて、1月27日は、世界的にハイテク株が売られた。これにより、エヌビディアは、時価総額で一時18％下落し、5927億ドル（約92兆円）が消失した。トランプ米大統領は27日、「ディープシークによる生成AIの発表が、競争に勝つためにさらに集中すべきだという米国AI業界への警鐘として受け止めるべきだ」と語った。\n\n米シリコンバレーのベンチャーキャピタリスト、マーク・アンドリーセン氏は1月26日のSNS「X」で、ディープシークの「R1モデル」は、AIの「スプートニク・モーメント」だと投稿した。アンドリーセン氏は、1950年代後半に、宇宙競争の開始を告げた旧ソビエト連邦による人工衛星の打ち上げに例え、米国が抱く危機感を表した。米紙「ワシントンポスト」によると、今回の開発費は約560万ドル（約8．7億円）で、米国製の1割程度とされる。ディープシークには、エヌビディア製の格落ちチップ「H800」 約2000個が、一方、米国製は最新型「H100」数万個が使用されている。\n\n主要テック企業3社は1月21日、米国におけるAIインフラの拡大を目指す新会社「スターゲート」を設立すると発表した。オープンAIのサム・アルトマン最高経営責任者（CEO）、ソフトバンクの孫正義CEO、オラクルのラリー・エリソン会長は同日、ホワイトハウスでトランプ大統領と共に設立を発表。同プロジェクトには今後、米国内で最大5000億ドル（約78兆円）の投資が予定されている。トランプ氏は、「世界的なテックの巨人が『スターゲート』を設立する。技術とAIは全部”Made in USA”にする」と述べた。計画には、全米20カ所のデータセンター建設も含まれている。最初となる約50万平方フィートのデータセンターは、米テキサス州に建設する予定となっている。\n\n米ホワイトハウスのレビット報道官は1月28日、ディープシークが国家安全保障に与える影

通常のスクレイピングなどと違い、内容のサマリーの生成といったこともしてくれていることがわかります。


## Firecrawl の extract の使い方

extract 関数は、スクレイピングしたテキストを、指定したスキーマに沿って抽出する関数です。
上記の scrape_url をよりデータ抽出に特化させたエンドポイントとなっています。

extract 関数の引数には、**URL をリスト形式で**、また、dict 形式で、AI に対するプロンプトと希望のアウトプットスキーマを指定します。


In [13]:
extract_result = firecrawl.extract(
    [target_url],
    {
        "prompt": "ニュース記事のタイトル、内容、サマリーを抽出してください。",
        "schema": NewsArticle.model_json_schema(),
    },
)

extract_result

{'success': True,
 'data': {'title': '【中国AIディープシーク】低コストで高性能“米政権に焦燥”技術覇権と安全保障は？（テレビ朝日系（ANN）） - Yahoo!ニュース',
  'content': '中国東部の浙江省杭州市に本社を置く新興企業「ディープシーク（DeepSeek）」が最新の人工知能（AI）モデルを発表し、世界の株式市場と米政権を揺るがした。同社製品は、米国のトップモデルに性能面で引けを取らず、低コストで実現したとしている。AI半導体大手の米エヌビディアなどの株価急落に直結した。この衝撃を受けて、1月27日は、世界的にハイテク株が売られた。これにより、エヌビディアは、時価総額で一時18％下落し、5927億ドル（約92兆円）が消失した。トランプ米大統領は27日、「ディープシークによる生成AIの発表が、競争に勝つためにさらに集中すべきだという米国AI業界への警鐘として受け止めるべきだ」と語った。\n\n米シリコンバレーのベンチャーキャピタリスト、マーク・アンドリーセン氏は1月26日のSNS「X」で、ディープシークの「R1モデル」は、AIの「スプートニク・モーメント」だと投稿した。アンドリーセン氏は、1950年代後半に、宇宙競争の開始を告げた旧ソビエト連邦による人工衛星の打ち上げに例え、米国が抱く危機感を表した。米紙「ワシントンポスト」によると、今回の開発費は約560万ドル（約8．7億円）で、米国製の1割程度とされる。ディープシークには、エヌビディア製の格落ちチップ「H800」 約2000個が、一方、米国製は最新型「H100」数万個が使用されている。\n\n主要テック企業3社は1月21日、米国におけるAIインフラの拡大を目指す新会社「スターゲート」を設立すると発表した。オープンAIのサム・アルトマン最高経営責任者（CEO）、ソフトバンクの孫正義CEO、オラクルのラリー・エリソン会長は同日、ホワイトハウスでトランプ大統領と共に設立を発表。同プロジェクトには今後、米国内で最大5000億ドル（約78兆円）の投資が予定されている。トランプ氏は、「世界的なテックの巨人が『スターゲート』を設立する。技術とAIは全部”Made in USA”にする」と述べた。計画には、全米20カ所のデータセンター建設も含まれている。最初となる約50

In [16]:
print("<title>", extract_result["data"]["title"], "</title>")
print("<summary>", extract_result["data"]["summary"], "</summary>")
print("<content>", extract_result["data"]["content"], "</content>")

<title> 【中国AIディープシーク】低コストで高性能“米政権に焦燥”技術覇権と安全保障は？（テレビ朝日系（ANN）） - Yahoo!ニュース </title>
<summary> 中国の新興企業ディープシークが最新のAIモデルを発表し、米国の株式市場に影響を与えた。ディープシークの製品は低コストで高性能とされ、エヌビディアの株価が急落。トランプ大統領はこの発表を米国AI業界への警鐘と捉え、AIインフラ拡大を目指す新会社「スターゲート」の設立も発表された。ディープシークのAI使用制限の動きが広がる中、国家安全保障への影響が懸念されている。 </summary>
<content> 中国東部の浙江省杭州市に本社を置く新興企業「ディープシーク（DeepSeek）」が最新の人工知能（AI）モデルを発表し、世界の株式市場と米政権を揺るがした。同社製品は、米国のトップモデルに性能面で引けを取らず、低コストで実現したとしている。AI半導体大手の米エヌビディアなどの株価急落に直結した。この衝撃を受けて、1月27日は、世界的にハイテク株が売られた。これにより、エヌビディアは、時価総額で一時18％下落し、5927億ドル（約92兆円）が消失した。トランプ米大統領は27日、「ディープシークによる生成AIの発表が、競争に勝つためにさらに集中すべきだという米国AI業界への警鐘として受け止めるべきだ」と語った。

米シリコンバレーのベンチャーキャピタリスト、マーク・アンドリーセン氏は1月26日のSNS「X」で、ディープシークの「R1モデル」は、AIの「スプートニク・モーメント」だと投稿した。アンドリーセン氏は、1950年代後半に、宇宙競争の開始を告げた旧ソビエト連邦による人工衛星の打ち上げに例え、米国が抱く危機感を表した。米紙「ワシントンポスト」によると、今回の開発費は約560万ドル（約8．7億円）で、米国製の1割程度とされる。ディープシークには、エヌビディア製の格落ちチップ「H800」 約2000個が、一方、米国製は最新型「H100」数万個が使用されている。

主要テック企業3社は1月21日、米国におけるAIインフラの拡大を目指す新会社「スターゲート」を設立すると発表した。オープンAIのサム・アルトマン最高経営責任者（CEO）、ソフトバンクの孫正義CEO、オラクルのラリー・エリソン会長

In [21]:
# 直接コピーしたテキストと合致するかのテスト

string = """中国東部の浙江省杭州市に本社を置く新興企業「ディープシーク（DeepSeek）」が最新の人工知能（AI）モデルを発表し、世界の株式市場と米政権を揺るがした。同社製品は、米国のトップモデルに性能面で引けを取らず、低コストで実現したとしている。AI半導体大手の米エヌビディアなどの株価急落に直結した。この衝撃を受けて、1月27日は、世界的にハイテク株が売られた。これにより、エヌビディアは、時価総額で一時18％下落し、5927億ドル（約92兆円）が消失した。トランプ米大統領は27日、「ディープシークによる生成AIの発表が、競争に勝つためにさらに集中すべきだという米国AI業界への警鐘として受け止めるべきだ」と語った。

米シリコンバレーのベンチャーキャピタリスト、マーク・アンドリーセン氏は1月26日のSNS「X」で、ディープシークの「R1モデル」は、AIの「スプートニク・モーメント」だと投稿した。アンドリーセン氏は、1950年代後半に、宇宙競争の開始を告げた旧ソビエト連邦による人工衛星の打ち上げに例え、米国が抱く危機感を表した。米紙「ワシントンポスト」によると、今回の開発費は約560万ドル（約8．7億円）で、米国製の1割程度とされる。ディープシークには、エヌビディア製の格落ちチップ「H800」 約2000個が、一方、米国製は最新型「H100」数万個が使用されている。

主要テック企業3社は1月21日、米国におけるAIインフラの拡大を目指す新会社「スターゲート」を設立すると発表した。オープンAIのサム・アルトマン最高経営責任者（CEO）、ソフトバンクの孫正義CEO、オラクルのラリー・エリソン会長は同日、ホワイトハウスでトランプ大統領と共に設立を発表。同プロジェクトには今後、米国内で最大5000億ドル（約78兆円）の投資が予定されている。トランプ氏は、「世界的なテックの巨人が『スターゲート』を設立する。技術とAIは全部”Made in USA”にする」と述べた。計画には、全米20カ所のデータセンター建設も含まれている。最初となる約50万平方フィートのデータセンターは、米テキサス州に建設する予定となっている。

米ホワイトハウスのレビット報道官は1月28日、ディープシークが国家安全保障に与える影響について、国家安全保障会議（NSC）が精査していると明らかにした。ディープシークが開発したAIの使用を制限する動きが、世界の企業や政府機関の間で広がりを見せている。ディープシークの生成AIについて、データ流出の懸念から、世界で数百に及ぶ企業などが職員に利用制限を課している。この懸念について、中国外務省は「企業の問題は企業に確認してほしい」としながらも、「中国は企業や個人に対して違法なデータの収集や保存を要求したことはない」と反論した。

自民党の小野寺五典政調会長は1月31日、ディープシークに尖閣諸島が日本の領土かと尋ねたところ、「中国固有の領土と事実と違う答えが返ってきた」と指摘した。小野寺氏は「当たり前のことをねじ曲げてしまうのがディープシークだと心配している。危ないのでダウンロードはやめていただきたい」と主張した。"""

string

'中国東部の浙江省杭州市に本社を置く新興企業「ディープシーク（DeepSeek）」が最新の人工知能（AI）モデルを発表し、世界の株式市場と米政権を揺るがした。同社製品は、米国のトップモデルに性能面で引けを取らず、低コストで実現したとしている。AI半導体大手の米エヌビディアなどの株価急落に直結した。この衝撃を受けて、1月27日は、世界的にハイテク株が売られた。これにより、エヌビディアは、時価総額で一時18％下落し、5927億ドル（約92兆円）が消失した。トランプ米大統領は27日、「ディープシークによる生成AIの発表が、競争に勝つためにさらに集中すべきだという米国AI業界への警鐘として受け止めるべきだ」と語った。\n\n米シリコンバレーのベンチャーキャピタリスト、マーク・アンドリーセン氏は1月26日のSNS「X」で、ディープシークの「R1モデル」は、AIの「スプートニク・モーメント」だと投稿した。アンドリーセン氏は、1950年代後半に、宇宙競争の開始を告げた旧ソビエト連邦による人工衛星の打ち上げに例え、米国が抱く危機感を表した。米紙「ワシントンポスト」によると、今回の開発費は約560万ドル（約8．7億円）で、米国製の1割程度とされる。ディープシークには、エヌビディア製の格落ちチップ「H800」 約2000個が、一方、米国製は最新型「H100」数万個が使用されている。\n\n主要テック企業3社は1月21日、米国におけるAIインフラの拡大を目指す新会社「スターゲート」を設立すると発表した。オープンAIのサム・アルトマン最高経営責任者（CEO）、ソフトバンクの孫正義CEO、オラクルのラリー・エリソン会長は同日、ホワイトハウスでトランプ大統領と共に設立を発表。同プロジェクトには今後、米国内で最大5000億ドル（約78兆円）の投資が予定されている。トランプ氏は、「世界的なテックの巨人が『スターゲート』を設立する。技術とAIは全部”Made in USA”にする」と述べた。計画には、全米20カ所のデータセンター建設も含まれている。最初となる約50万平方フィートのデータセンターは、米テキサス州に建設する予定となっている。\n\n米ホワイトハウスのレビット報道官は1月28日、ディープシークが国家安全保障に与える影響について、国家安全保障会議（NSC）が精査していると明らかにした。ディー

In [22]:
extract_result["data"]["content"] == string

True

ということで、本当に抽出したいデータが決まっている場合は、 extract 関数のほうが便利そうです。


## まとめ ＆ Firecrawl の Extract API を活用したアプリの実例

Firecrawl の API は、ウェブページのテキストを抽出する際に非常に便利な API でした。

実はしばらく自分でスクレイピングの部分を実装しようとしていたのですが、時間コストも考えるとこういったライブラリのメリットはかなり大きいですね。

ついでに！
いろんなコードを参照しながら、Firecrawl の Extract API を使って小さなアプリも作ってみたので合わせてご活用ください。
求人リストから情報を抽出し、履歴書に対して一番相性の良い求人を提案してくれるアプリです。

https://github.com/alexweberk/ai-jobfinder

以上、お読みいただきありがとうございます。少しでも参考になればと思います。

もし似たようなコンテンツに興味があれば、フォローしていただけると嬉しいです：

- [note](https://note.com/alexweberk/) と
- [Twitter](https://twitter.com/alexweberk)

https://twitter.com/alexweberk
